In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

i = 0
for dirname, _, filenames in os.walk('/kaggle/input'):
    
    for filename in filenames:
        print(os.path.join(dirname, filename))
        i += 1
        if i > 5:
            break
    if i > 5:
        break
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from os import listdir
from os.path import isfile, join

path = '/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/'
onlydirs = [f for f in listdir(path) if isfile(join(path, f))]
len(onlydirs)

In [ ]:
from os import listdir
from os.path import isfile, join
import pydicom
 
examples = ['00000','00178']
for ex in examples: 
 
    p1, p2, p3, p4 = 'FLAIR', 'T2w', 'T1w', 'T1wCE'
    base_path = f'/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{ex}'
 
    print(f'Образец №{ex}:')
    for p in [p1, p2, p3, p4]:
        onlyfiles = [f for f in listdir(f'{base_path}/{p}/') if isfile(join(f'{base_path}/{p}/', f))]
        img = pydicom.read_file(f'{base_path}/{p}/{onlyfiles[0]}').pixel_array
        print(f'Количество файлов в типе {p}: {len(onlyfiles)}, примеры файлов: {onlyfiles[:2]}', 
             f'разрешение: {img.shape}')

In [ ]:
import os
#for file management
import json
#for standardized data storage
import glob
#for precise file selection(low verbosity)
import random
#random amount generator
import collections
#has premade datastructure objects that can be implemented
import numpy as np
#linear algebra
import pandas as pd
#succinct array and data handling
import pydicom
#c based dicom modulation tool
from pydicom.pixel_data_handlers.util import apply_voi_lut
#pixel handler that uses voi_lut to grab pixel data from within the frame of a window
import cv2
#image data handler
import matplotlib.pyplot as plt
#data visualization library
import seaborn as sns
#data visualization library


# /kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00162/T2w/Image-4.dcm

In [ ]:
train_df = pd.read_csv("/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")
train_df.head()

In [ ]:
train_df['FLAIR_path'] = '/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/' + train_df['BraTS21ID'].astype(str).str.zfill(5) + '/FLAIR/'
train_df['T1w_path'] = '/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/' + train_df['BraTS21ID'].astype(str).str.zfill(5) + '/T1w/' 
train_df['T2w_path'] = '/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/' + train_df['BraTS21ID'].astype(str).str.zfill(5) + '/T2w/' 
train_df['T1wCE_path'] = '/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/' + train_df['BraTS21ID'].astype(str).str.zfill(5) + '/T1wCE/' 


In [ ]:
train_df.info()

In [ ]:
train_df.head()

In [ ]:
def rescale(arr):
    arr_min = arr.min()
    arr_max = arr.max()
    if (arr_max - arr_min) == 0:
        return arr
    return (arr - arr_min) / (arr_max - arr_min)

In [ ]:
import matplotlib.pyplot as plt
import imageio
from IPython.display import Image

def visualize_data_gif(images):
    imageio.mimsave("/kaggle/working/1.gif", images, duration=5.0 / images.shape[0])
    return Image(filename="/kaggle/working/1.gif", format='png')

In [ ]:
import tensorflow as tf
import re

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
class MRIDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, X_col, y_col, batch_size,
                 input_size= (256, 256), depth_size=64,
                 shuffle=True):
    
        self.df = df.copy()
        self.X_col = X_col
        self.y_col = y_col
        self.depth_size = depth_size
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.n = len(self.df)
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
    
    def __get_input(self, path, target_size):
    
        scan3d = None
        onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
        filepatt = 'Image-{}.dcm'
        digits = [int(re.search('\d+',i).group()) for i in listdir(path) if re.match(filepatt.format('\d+\\'),i)]
        digits.sort()
        onlyfiles = [filepatt.format(dig) for dig in digits]
        
        center = len(onlyfiles) // 2
        left = max(0, center - (self.depth_size // 2))
        right = min(len(onlyfiles), center + (self.depth_size // 2))
        onlyfiles = onlyfiles[left: right]
        if len(onlyfiles) < self.depth_size:
            img_shape = pydicom.read_file(f'{path}{onlyfiles[0]}').pixel_array.shape
            add_z = self.depth_size - len(onlyfiles)
            scan3d = np.zeros((add_z, target_size[0], target_size[1],1))
        
            
        scans = []
        for f in onlyfiles:
            img = pydicom.read_file(f'{path}{f}')
            img = img.pixel_array
            img = self._rescale(img)
            img = np.expand_dims(img, axis=-1)
            img = tf.image.resize(img,(target_size[0], target_size[1])).numpy()
            
            img = self._normalize(img)
            scans.append(img)
        
        if scan3d is not None:
            return np.concatenate([np.array(scans), scan3d]) 
        else:
            return np.array(scans)
    
    def _rescale(self, arr):
        arr_min = arr.min()
        arr_max = arr.max()
        if (arr_max - arr_min) == 0:
            return arr
        return (arr - arr_min) / (arr_max - arr_min)
    
    def _normalize(self, arr):
        img = arr - arr.mean()
        # divide by the standard deviation (only if it is different from zero)
        if np.std(img) != 0:
            img = img / np.std(img)
        return img
    
    def __get_data(self, batches):
        if self.X_col is None:
            PATHS = ['FLAIR_path', 'T1w_path', 'T2w_path', 'T1wCE_path']
            X_batch = []
            for p in PATHS:
                batch_part_path = batches[p]
                X_batch.append(np.asarray([self.__get_input(x,  self.input_size) for x in batch_part_path]))
            y_batch = batches[self.y_col].values
            X_batch = np.concatenate(X_batch, axis=4)
            
        else:
            path_batch = batches[self.X_col]
            X_batch = np.asarray([self.__get_input(x,  self.input_size) for x in path_batch])
            y_batch = batches[self.y_col].values
        return X_batch, y_batch
    
    def __getitem__(self, index):
#         print('Star_loading')
        batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)
#         print(X.shape)
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size     

In [ ]:
class MRIDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, X_col, y_col, batch_size,
                 input_size= (256, 256), depth_size=64,
                 shuffle=True):
    
        self.df = df.copy()
        self.X_col = X_col
        self.y_col = y_col
        self.depth_size = depth_size
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.n = len(self.df)
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
    
    def __get_input(self, path, target_size):
        pass
    
    def _rescale(self, arr):
        pass
    
    def _normalize(self, arr):
        pass
    
    def __get_data(self, batches):
        pass

    
    def __getitem__(self, index):
        pass
   
    
    def __len__(self):
        return self.n // self.batch_size   

In [ ]:
!pip install efficientnet-3D
!pip install keras_applications 
# import efficientnet_3D.keras as efn 
import efficientnet_3D.tfkeras as efn

In [ ]:
import imageio
from IPython.display import Image

def visualize_data_gif(images):
    imageio.mimsave("/kaggle/working/1.gif", images, duration=5.0 / images.shape[0])
    return Image(filename="/kaggle/working/1.gif", format='png')

In [ ]:
gen = MRIDataGenerator(train_df, э, 'MGMT_value', 10, (256, 256), 64, True)
iterator = iter(gen)

In [ ]:
data = next(iterator)[0] * 255
images = []
for i in range(4):
    a = data[:,:,:,:, i]
    images.append(np.stack([a,a,a], axis=4).reshape((*a.shape[:4], 3)))


In [ ]:
val = np.concatenate(images, axis=3)
visualize_data_gif(val[5])

In [ ]:
visualize_data_gif(val[0])

In [ ]:
a = images[:,:,:,:, i]
z = np.stack([a,a,a], axis=4).reshape((*a.shape[:4], 3))
visualize_data_gif(z[0])

In [ ]:
next(iterator)[0].shape

In [ ]:
import efficientnet_3D.tfkeras as efn

In [ ]:
depth = 64
resolution = (192, 192)
batches = 8
gen = MRIDataGenerator(train_df, 'FLAIR_path', 'MGMT_value', batches, resolution, depth, True)


In [ ]:
#check
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense
# model = Sequential()
# model.add(Conv3D(16, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(32, 150, 150, 1)))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))
# model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))
# model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))
# model.add(Flatten())
# model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
# model.add(Dense(1, activation='sigmoid'))


In [ ]:
with tf.device('/gpu:0'):
    
    from tensorflow.keras import Sequential
    from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense
    model = Sequential()
    model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(64,192, 192, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(
        optimizer='adam', 
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.BinaryAccuracy()]
        )
    
    history = model.fit(gen, steps_per_epoch = batches, 
                              verbose=1,
                              epochs = 10
                       )

In [ ]:
with tf.device('/gpu:0'):
    
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(64,64,32,1)),
        tf.keras.layers.Conv3D(3, kernel_size=1, padding='same'),
        efn.EfficientNetB0(include_top=False, input_shape=(64,64,32,3), pooling='avg'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam', 
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC()]
        )
    
    history = model.fit(gen, steps_per_epoch = batches, 
                              verbose=1,
                              epochs = 3
                       )

In [ ]:
input_layer = tf.keras.layers.InputLayer(input_shape=(depth, resolution[0], resolution[1], 1))

In [ ]:
model = efn.EfficientNetB0(input_shape=(depth, resolution[0], resolution[1], 1), weights=None)

In [ ]:
x = tf.keras.layers.Flatten()(model.output)
out = tf.keras.layers.Dense(1, activation="sigmoid")(x)

In [ ]:
new_model = tf.keras.Model(inputs=model.input, outputs=out)

In [ ]:
# new_model.summary()

In [ ]:
new_model.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
history = new_model.fit(gen, steps_per_epoch = batches, 
                              verbose=1,
                              epochs = 3)

In [ ]:
plt.plot(history.history['loss'])
plt.ylabel("loss")
plt.xlabel("epoch")
plt.title("Training Loss Curve")
plt.show()